In [3]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn

In [4]:
# %load transformer_model.py 

import os
import sys

PROJECT_ROOT = os.path.abspath("..")  # common/의 상위 폴더
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from main import TransformerModel

In [5]:
path = "../data/en_fr_small.tsv"
df = pd.read_csv(path, sep="\t")
df

,en,fr
0,Hello.,Bonjour.
1,How are you?,Comment vas-tu ?
2,I like coffee.,J'aime le café.
3,This is a book.,Ceci est un livre.
4,He is reading a newspaper.,Il lit un journal.
5,She lives in Paris.,Elle habite à Paris.
6,We are traveling tomorrow.,Nous voyageons demain.
7,They play football every weekend.,Ils jouent au football chaque week-end.
8,Thank you very much.,Merci beaucoup.
9,See you later.,À plus tard.


In [6]:
valid_path = "../data/en_fr_small_valid.tsv"
valid_df = pd.read_csv(valid_path, sep="\t")
valid_df

,en,fr
0,Good morning.,Bonjour.
1,I am learning French.,J'apprends le français.
2,Do you understand?,Comprends-tu ?
3,We need more time.,Nous avons besoin de plus de temps.
4,The weather is nice today.,Il fait beau aujourd'hui.


In [11]:
print(len(df))
print(df.shape)

10
(10, 2)


In [ ]:
item = df.iloc[0]
item["en"], item["fr"]

('Hello.', 'Bonjour.')

In [27]:
batch_size = 4
shuffle = True

In [14]:
train_loader = DataLoader(
    df, batch_size=4
)
train_loader.dataset

,en,fr
0,Hello.,Bonjour.
1,How are you?,Comment vas-tu ?
2,I like coffee.,J'aime le café.
3,This is a book.,Ceci est un livre.
4,He is reading a newspaper.,Il lit un journal.
5,She lives in Paris.,Elle habite à Paris.
6,We are traveling tomorrow.,Nous voyageons demain.
7,They play football every weekend.,Ils jouent au football chaque week-end.
8,Thank you very much.,Merci beaucoup.
9,See you later.,À plus tard.


In [17]:
valid_loader = DataLoader(
    valid_df, batch_size=4
)
valid_loader.dataset

,en,fr
0,Good morning.,Bonjour.
1,I am learning French.,J'apprends le français.
2,Do you understand?,Comprends-tu ?
3,We need more time.,Nous avons besoin de plus de temps.
4,The weather is nice today.,Il fait beau aujourd'hui.


In [7]:
model = TransformerModel(d_model=128, num_heads=8, num_layers=2)#.to(device)

In [ ]:
# fit ====
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
pad_id = model.tokenizer.pad_token_id
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_id)

# 같은 데이터셋을 3번 학습시킨다.
for epoch in range(3):
    model.train()
    for i in range(len(df)):
        batch = df.iloc[i]
        print(batch)
        src = model.tokenizer(batch["en"], return_tensors="pt", padding=True)#.to(device)
        tgt = model.tokenizer(batch["fr"], return_tensors="pt", padding=True)#.to(device)

        decoder_inputs = tgt["input_ids"][:, :-1]
        decoder_mask = tgt["attention_mask"][:, :-1]
        labels = tgt["input_ids"][:, 1:]

        logits = model(
            src["input_ids"],
            decoder_inputs,
            src_attention_mask=src["attention_mask"],
            tgt_attention_mask=decoder_mask,
        )
        loss = loss_fn(logits.reshape(-1, logits.size(-1)), labels.reshape(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


Positional Encoding :  tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  7.6172e-01,  ...,  1.0000e+00,
          1.1548e-04,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  9.8705e-01,  ...,  1.0000e+00,
          2.3096e-04,  1.0000e+00],
        ...,
        [ 6.1950e-02,  9.9808e-01,  8.1507e-01,  ...,  9.9770e-01,
          5.8745e-02,  9.9827e-01],
        [ 8.7333e-01,  4.8714e-01,  9.6940e-01,  ...,  9.9769e-01,
          5.8860e-02,  9.9827e-01],
        [ 8.8177e-01, -4.7168e-01,  4.4109e-01,  ...,  9.9768e-01,
          5.8975e-02,  9.9826e-01]])
en      Hello.
fr    Bonjour.
Name: 0, dtype: object


TypeError: TransformerModel.forward() got an unexpected keyword argument 'src_attention_mask'

In [ ]:
# predict ====
model.eval() # eval : 드롭아웃·레이어 정규화 등 학습 전용 동작을 꺼주는 함수
model.predict("Hello")

AttributeError: 'TransformerModel' object has no attribute 'predict'